## modeling of estimation on sqft price

In [6]:
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1

import os,sys
import numpy as np
import pandas as pd

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), '../..', 'src')
sys.path.append(src_dir)

%aimport plot_utils
from plot_utils import *
%aimport preprocessing
from preprocessing import *
%aimport data_source
from data_source import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### DataSource 
* This module loads data from database
* Auto update views by executing database/preprocess.sql

In [2]:
ds = DataSource(host='localhost',port=5432, username='postgres', password='', database='sdra')
ds.show_views()
ds.update_views()

connect to database postgresql://postgres:@localhost:5432/sdra
['properties', 'property_features', 'transactions', 'property_addresses', 'property_address_transactions', 'property_estimate']
execute  c:\wenyan\dse_capstone\database\preprocess.sql


### Preprocess
* This module will deliver the valid dataset with finished features

In [8]:
ds = DataSource(host='localhost',port=5432, username='postgres', password='', database='sdra')
ds.show_views()
ds.update_views()

df_transaction = ds.get_view_df('property_address_transactions')
df_property = ds.get_view_df('property_addresses')
print(df_transaction.shape)
print(df_property.shape)

connect to database postgresql://postgres:@localhost:5432/sdra
['properties', 'property_features', 'transactions', 'property_addresses', 'property_address_transactions', 'property_estimate']
execute  c:\wenyan\dse_capstone\database\preprocess.sql
(1512560, 16)
(709325, 7)


In [10]:
pp = Preprocess(df_transaction, df_property)

### dataset for modeling
Preprocess.dataset(feature='delivered', valid=True)
* suggest using: feature='delivered', otherwise all features including non-solid features will be included
* suggest using: valid=True, otherwise outliers and invalid feature values will be  included
* date range can be specified

In [11]:
# records of all years
df_all_valid = pp.dataset(feature='delivered', valid=True)

transforming view
transforming pool
transforming date
transforming year_built
transforming sold_year
transforming sold_age
making sqft_zip_avg
making sqft_price_zip_avg
making sold_price_zip_avg
making sale_count_zip
making prop_count_zip


In [12]:
# records of last year
df_last_year_valid = pp.dataset(feature='delivered', valid=True, date=('2016-10-01', '2017-10-01'))
df_last_year_valid.shape

transforming view
transforming pool
transforming date
transforming year_built
transforming sold_year
transforming sold_age
making sqft_zip_avg
making sqft_price_zip_avg
making sold_price_zip_avg
making sale_count_zip
making prop_count_zip


(37881, 9)

In [15]:
df_last_year_valid[:3]

,view,sqft_price,pool,sqft_price_zip_avg,sqft,num_bed,sold_price_zip_avg,num_bath,sqft_zip_avg
0,1,243,0,257.241848,3580,4.0,412059.782609,2.0,1662.940217
1,0,213,0,257.241848,2785,3.0,412059.782609,2.0,1662.940217
2,0,260,0,257.241848,1822,4.0,412059.782609,2.0,1662.940217


In [13]:
# records of last 2 years
df_last_2year_valid = pp.dataset(feature='delivered', valid=True, date=('2015-10-01', '2017-10-01'))
df_last_2year_valid.shape

transforming view
transforming pool
transforming date
transforming year_built
transforming sold_year
transforming sold_age
making sqft_zip_avg
making sqft_price_zip_avg
making sold_price_zip_avg
making sale_count_zip
making prop_count_zip


(76689, 9)

In [14]:
# records of last 3 years
df_last_3year_valid = pp.dataset(feature='delivered', valid=True, date=('2014-10-01', '2017-10-01'))
df_last_3year_valid.shape

transforming view
transforming pool
transforming date
transforming year_built
transforming sold_year
transforming sold_age
making sqft_zip_avg
making sqft_price_zip_avg
making sold_price_zip_avg
making sale_count_zip
making prop_count_zip


(111380, 9)

### regression model
* target: sqft_price
* try various features combination: ...
* try different dataset size: last 1 year, 2 years, ...
* try different models: LinearRegression, RandomForestRegression, ...